In [16]:
from configparser import ConfigParser

class ConfigParserHelper:
    """
    Helper class to make job easier for config parser methods.
    """
    def __init__(self, file_path: str):
        self.file_path = file_path
        self.config = ConfigParser()

    def load(self):
        """
        Load the config file into the config parsher object.
        """
        self.config.read(self.file_path)

    def has_section(self, section: str):
        """
        Check the config file has the given section or not.
        """
        if section in self.config.sections():
            return True
        return False

class ConfigSelector:
    """
    Decides which config class to return depending on deployment mode.
    """

    def __new__(cls):
        obj = ConfigParserHelper.__new__(ConfigParserHelper)
        obj.__init__(cls.FILE_PATH)
        return obj


class SparkConfig(ConfigSelector):
    """
    Config parser for the spark config file.
    """
    FILE_PATH = "/home/pavithra/projects/MLOPS_learning/sample_project/core/config/spark_config.cfg"

c = SparkConfig()

In [19]:
c.get("conneciton")

AttributeError: 'ConfigParserHelper' object has no attribute 'get'

In [ ]:
import os

from configparser import RawConfigParser

from insights_common.configs.config import Config
from insights_common.logger import getLogger

logger = getLogger("insights-ids-core")


class FileConfig(Config):

    def _init_(self, default_dir: str, filename: str, persisted_dir: str, load: bool = True):
        """
        Initialize an FileConfig object.

        :param default_dir: Absolute path to location/directory of default configuration file.
        :param filename: Name of configuration file (no path).
        :param persisted_dir: Path within ids-core container where the persistent config is stored.
        """
        super()._init_(load)
        self.DEFAULT_CONFIG_FILE = os.path.join(default_dir, filename)
        self.PERSISTENT_CONFIG_FILE = os.path.join(persisted_dir, filename)
        self.source = None  # keep track of which file has actually been loaded
        if self.load_cfg:
            self.load()

    def exists(self) -> bool:
        """
        Check if a config file exists.

        :return: True if config file exists on system (either default or persistent).
        """
        return self.configured() or os.path.isfile(self.DEFAULT_CONFIG_FILE)

    def configured(self) -> bool:
        """
        Check if a config has been saved.

        :return: True if the persistent config file exist, else False.
        """
        return os.path.isfile(self.PERSISTENT_CONFIG_FILE)

    def load(self):
        """
        Re-load the config from disk or if it doesn't exist load the default config file.
        """
        if self.configured():
            self.source = self.PERSISTENT_CONFIG_FILE
        else:
            self.source = self.DEFAULT_CONFIG_FILE
        self._config = RawConfigParser()
        self._config.read(self.source)

    def save(self):
        """
        Save config to persistent location and spread to all hosts
        """
        with open(self.PERSISTENT_CONFIG_FILE, "w", encoding="utf-8") as config_file:
            self._config.write(config_file)